# Quelques notions de parallèlisme

Les enjeux principaux du calcul scientifique sont
- la précision des modèles physiques mis en oeuvre dans la simulation : cela requiert évidemment une très bonne connaissance de la physique des phénomènes étudiés, et des équations comportementales correspondantes,
- la précision des modèles numériques de résolution : cela requiert une bonne connaissance dans le domaine des mathématiques appliquées, permettant de passer d'une représentation formelle et continue à une formulation numérique discrète,
- la précision des calculs numériques dans les algorithmes de résolution : ce point est souvent complémentaire du point précédent, l'objectif du numéricien étant l'algorithme le plus précis possible, au sens méthodologique et au sens purement numérique du terme,
- l'autre point, qui fait l'objet de cette séance, est le temps de calcul qui peut rapidement devenir critique voire bloquant.

Il y a quelques années encore, il était possible de se dire que les problèmes de temps de calcul d'aujourd'hui seront résolus à condition d'attendre encore un peu l'évolution des technologies des microprocesseurs. C'est la fameuse loi de Gordon Moore, un des fondateurs d'Intel.

Au milieu des années 60, Moore énonce une loi empirique selon laquelle le nombre de transistors doublera tous les 18 mois sur une même surface semi-conductrice, les performances des microprocesseurs étant évidemment corrélées au nombre de transistors disponibles.

Cette loi a été réévaluée au milieu des années 70, avec un doublement tous les deux ans.

Moore prévoit une évolution similaire pour les fréquences d'horloge des microprocesseurs, la fréquence d'horloge étant là encore directement corrélée à la puissance de calcul.

Cette loi a globalement été plutôt bien suivie jusqu'à récemment. On observe cependant depuis quelques années un phénomène de saturation par rapport à cette loi de Moore :
- pour ce qui est du nombre de transistors, on commence à atteindre certaines limites dans le domaine de la miniaturisation des composants
- pour ce qui est des fréquences d'horloge, la puissance dissipée évolue comme le cube de cette fréquence, ce qui pose des problèmes majeurs en terme de refroidissement des unités centrales.

Aujourd'hui l'évolution des microprocesseurs a donc tendance à se ralentir. En revanche on dispose d'une marge forte de progression par la multiplication des machines.

Ainsi, ce que l'on peut difficilement gagner par des performances accrues des microprocesseurs on va le gagner en multipliant les ressources matérielles utilisées pour le calcul, ce qui nous amène à la notion de parallèlisme.

Faire du calcul parallèle revient à partager le calcul complet entre plusieurs processeurs ou plusieurs machines distinctes, afin de réduire le temps de calcul global. On peut ainsi espérer qu'en répartissant un calcul entre 4 machines différentes on arrivera à réduire le temps d'un facteur 4. En réalité les choses sont évidemment un peu plus complexes ...

Le calcul parallèle est accessible même avec nos ordinateurs personnels, qui aujourd'hui sont tous multi-coeurs. Même des machines dites "grand public" peuvent facilement être quadri-coeurs voire plus. Cela nous permet de disposer de plusieurs unités de traitement, qui peuvent être utilisées en parallèle plutôt que de manière séquentielle. On est alors sur un schéma de parallèlisme à mémoire partagée : on utilise une seule machine physique qui dispose de un ou plusieurs microprocesseurs, chaque processeur étant lui-même multi-coeurs.

Sous réserve de pouvoir disposer du matériel nécessaire, on peut également faire appel à plusieurs machines interconnectées par un réseau rapide (on parle souvent de réseau infini-bande). les techniques de parallèlisme sont un peu plus complexes car il est nécessaire d'échanger des données entre les différentes machines. C'est typiquement le genre d'architecture que l'on va trouver dans les clusters et grands centres de calcul.

En France les super-calculateurs sont gérés par le GENCI (Grand Equipement National de Calcul Intensif) avec plusieurs centres tels que
- le CINES, Centre Informatique National de l’Enseignement Supérieur à Montpellier
- l’Institut du développement et des ressources en informatique scientifique du CNRS (IDRIS) à Orsay (en particulier le calculateur Jean Zay)
- le Très Grand Centre de Calcul du CEA (TGCC) à Bruyères-le-Châtel
- le calculateur Myria au CRIANN (Centre Régional Informatique et d'Applications Numériques de Normandie)

La liste ci-dessus ne se veut pas exhaustive. On trouve également des structures à l'échelle de l'Union Européenne, avec des programmes de financements européens.

De manière générale, ces gros centres de calcul sont d'accès gratuit pour les structures académiques (laboratoires de recherche, universités, écoles), avec également un accès possible pour les entreprises qui ont besoin de ressources importantes en calcul numérique.

Le passage au calcul parallèle n'est cependant pas simple, il nécessite déjà d'évaluer si le calcul est parallèlisable ou non, sachant que certains calculs peuvent difficilement se faire autrement que de manière séquentielle. Reste ensuite l'implémentation, qui peut nécessiter une réécriture des codes pour qu'ils puissent être adaptés à une structure parallèle. La recherche d'erreurs est également souvent plus complexe dans une architecture parallèle, avec des erreurs difficilement reproductibles et très spécifiques à la manière dont le calcul est parallèlisé.

Il est ainsi très important de vérifier qu'un calcul parallèle donne les mêmes résultats que le même calcul réalisé de manière séquentielle.

Une autre façon de faire du calcul parallèle consiste à utiliser les GPU - Graphical Processing Unit. A l'origine les GPU ont été développés principalement pour les consoles de jeux, permettant de donner une forte puissance de calcul à la carte graphique, assurant ainsi une bonne fluidité dans l'affichage des scènes des jeux. Ces cartes GPU sont aujourd'hui beaucoup utilisées pour faire du calcul scientifique intensif, avec des cartes pouvant comporter plusieurs milliers d'unités de traitement.

Il existe une grande variété d'approches pour faire du calcul parallèle. Lors de cette séance nous allons voir comment on peut mettre en oeuvre une structure de calcul parallèle à mémoire partagée.


## Premier exemple de calcul intensif

A titre d'exemple de calcul intensif, on peut calculer la décomposition en facteurs premiers d'un nombre entier $n$. Sans chercher à optimiser l'algorithme de calcul, on peut utiliser le schéma de calcul suivant :
- une fonction récursive avec comme paramètres la liste des diviseurs déjà trouvés et la valeur résiduelle de $n$
- cette fonction teste les diviseurs possibles entre 2 et $n$
- si $i$ divise $n$, on ajoute $i$ à la liste des diviseurs trouvés, puis on appelle la fonction récursivement en remplaçant $n$ par $n/i$.

Ecrire cette fonction de calcul.


Evaluer le temps de calcul nécessaire pour la décomposition en facteurs premiers de tous les entiers entre 1 et 100000

### Parallèlisation du calcul

Une manière simple de faire du calcul parallèle consiste à utiliser la composante Pool du module multiprocessing. Votre fonction de décomposition en facteurs premiers recevant un seul et unique paramètre $n$, la fonction pool.map() permet de manière très simple de parallèliser l'appel d'une fonction utilisateur pour une liste importante de valeurs possible du paramètre $n$.

Appliquer cette approche pour décomposer en facteurs premiers tous les entiers entre 1 et 100000. Comparer le temps de calcul avec la version précédente non parallèle.

Il sera ici utile et intéressant de regarder le taux de charge CPU de l'ordinateur pendant que les boucles de calcul s'exécutent.

## Second exemple - la multiplication de 2 matrices

La multiplication de 2 matrices est un bon exemple de calcul qui peut facilement être mené de manière parallèle car le calcul des différentes lignes de la matrice produit est totalement indépendant entre les lignes.

Ecrire un programme permettant de 
- générer deux matrices $A$ et $B$ aléatoires de taille $N\times N$
- calculer le produit $C=AB$ par un calcul séquentiel
- calculer le produit $D=AB$ par un calcul parallèle
- vérifier l'égalité $C=D$

Comparer les temps de calcul entre la version séquentielle et la version parallèle, en fonction de la taille $N$
